<a href="https://colab.research.google.com/github/CubatLin/Data-Science-Project/blob/main/Transformers%20-%20CKIP/src/Model_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tqdm import tqdm
import pickle
import re
import os 
import numpy as np
import pickle

os.getcwd()

'/content'

In [2]:
#---Main---#
class Tagging:
    def __init__(self,name,data):
        self.name = name; self.data = data
        self.Regex_func(); self.NER_func(); self.WsPos_func()
        self.MainProcess_WordFreq()

  #--先把爬到的資料清乾淨
    def Regex_func(self):
        t = []
        for l in self.data: 
            #清掉爬蟲日期
            l = re.sub(r"[(A-Za-z) (0-9), (0-9) —|(0-9)年(0-9)月(0-9)日]+",'',l)
            #append結果
            t.append(re.sub(r"[^\u4e00-\u9fa5|^(0-9)]+",' ',l) )
        self.data = t

  #--把原本資料中NER挑出來，並去除掉
    def NER_func(self):
        ner = ner_driver(self.data); ner_s = []
        for i in range(len(ner)):
            ner_s.extend(x.word for x in ner[i] if x.word not in ner_s)
        for i in range(len(self.data)):
            tmp_s = self.data[i]
        for x in ner_s:
            tmp_s = tmp_s.replace(x,"")
        self.data[i] = tmp_s 

    def WsPos_func(self):
        ws = ws_driver(self.data); pos = pos_driver(ws) #詞性  
        self.ws_res ,self.pos_res = self.pack_ws_pos_sentece(ws,pos)

  #input: list in list
    def pack_ws_pos_sentece(self,sentence_ws, sentence_pos): #要加self的原因是告訴這個class這個function有這兩個引數
        assert len(sentence_ws) == len(sentence_pos)
        res_ws = []; res_pos = [] 
        pos_set = set(['Na','Nb','Nc','VC','Vn']) #定義保留的詞性
        for word_ws, word_pos in zip(sentence_ws, sentence_pos):
            # Two-Pointer
            l_idx = 0; r_idx=len(word_pos)-1
            while l_idx<=r_idx:
                if  word_pos[l_idx] in pos_set:
                    word_ws[l_idx] , word_ws[r_idx] =  word_ws[r_idx] , word_ws[l_idx]
                    word_pos[l_idx] , word_pos[r_idx] =  word_pos[r_idx] , word_pos[l_idx]
                    r_idx-=1
                else: l_idx+=1
            res_ws.append(word_ws[l_idx:])
            res_pos.append(word_pos[l_idx:])
        return res_ws , res_pos
  
  #--flattern list :遍歷所有文字, 留下討論度高的
    def MainProcess_WordFreq(self,n_freq=15):
        dic ={}
        for l in self.ws_res:
            for x in l:
                if x not in dic : dic[x]=1
                else : dic[x]+=1
        tmp =[]; tmp_name = []
        for x in dic.items():
        #清除空白 & 特店/特色等字串
            if re.match(r"[^ \s*|^特.|^品.|^理.|^精神.|^價值.|^故.|^形.]",x[0]):
                if x[1]>n_freq and len(x[0])>1:
                    tmp.append((x[0],x[1]))
                    tmp_name.append(x[0])
        self.tagging_list = tmp_name
        self.freq_list = tmp

#### Create Data

In [6]:
def import_data_from_drive(id):
  !wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{id} -O- \
  | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt && wget --content-disposition --load-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{id}'&confirm='$(<confirm.txt)


In [7]:
import_data_from_drive('1vBGSXzJ0z2xtq-df6kXEgfhuQWqFp9UX')
import_data_from_drive('1-JXf_APs0tBrWSTfPp1Os6B4nNHKbUYh')
import_data_from_drive('1-7wznO1o7jY5GAP_5Bz_03UMpsMPJcou')
import_data_from_drive('1-4oy2lqxtEV_n7ROIDizkHRAiCLaToOC')

--2022-01-26 09:24:46--  https://docs.google.com/uc?export=download&id=1vBGSXzJ0z2xtq-df6kXEgfhuQWqFp9UX
Resolving docs.google.com (docs.google.com)... 142.251.107.138, 142.251.107.101, 142.251.107.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.107.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mg7sv59pt5sfo6d55l20h93pthr2mqe4/1643189025000/07903891748575983860/*/1vBGSXzJ0z2xtq-df6kXEgfhuQWqFp9UX?e=download [following]
--2022-01-26 09:24:46--  https://doc-08-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mg7sv59pt5sfo6d55l20h93pthr2mqe4/1643189025000/07903891748575983860/*/1vBGSXzJ0z2xtq-df6kXEgfhuQWqFp9UX?e=download
Resolving doc-08-1s-docs.googleusercontent.com (doc-08-1s-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to doc-08-1s-docs.googleusercontent.com (doc-08

In [8]:
load_pkl_path = '/content'
max_num = max([int(x.split('.')[0][1:]) for x in os.listdir(load_pkl_path) if 'pickle' in x ])

In [11]:
def read_brand_pkl(i):
    try:
        with open(load_pkl_path + f'/x{i}.pickle', 'rb') as f:
            globals()[f'x{i}'] = pickle.load(f)
        return f'x{i}'
    except: 
        print(f'Fail: x{i}')

In [12]:
%%time 
success_list=[]
for i in tqdm(range(max_num+1)):
    success_list.append(read_brand_pkl(i))
    
success_list = [x for x in success_list if x != None]

100%|██████████| 4/4 [00:00<00:00, 842.44it/s]

CPU times: user 7.49 ms, sys: 2.88 ms, total: 10.4 ms
Wall time: 10.7 ms


In [25]:
x0.MainProcess_WordFreq(7)
x1.MainProcess_WordFreq(7)
x2.MainProcess_WordFreq(7)
x3.MainProcess_WordFreq(7)

In [26]:
x0.name

'誠品書店'

In [27]:
x0.tagging_list

['書店',
 '風格',
 '誠品',
 '城市',
 '臺灣',
 '文具',
 '創意',
 '設計',
 '生活',
 '藝術',
 '人文',
 '臺北',
 '文化',
 '百貨',
 '模式',
 '分店',
 '經營',
 '小時',
 '信義',
 '商品',
 '基隆',
 '閱讀',
 '核心',
 '書籍',
 '敦南店']

#### Gensim Model

In [3]:
import gensim 

#Load Model
gensim.__version__

'3.6.0'

In [4]:
%%time
import_data_from_drive('1zcTm-OG-oRZSw0ef-V_nY8RsSBgWn1Pb')

--2022-01-26 09:22:48--  https://docs.google.com/uc?export=download&id=1zcTm-OG-oRZSw0ef-V_nY8RsSBgWn1Pb
Resolving docs.google.com (docs.google.com)... 108.177.12.113, 108.177.12.100, 108.177.12.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

-                       [ <=>                ]   3.19K  --.-KB/s    in 0s      

2022-01-26 09:22:49 (75.2 MB/s) - written to stdout [3269]

--2022-01-26 09:22:49--  https://docs.google.com/uc?export=download&id=1zcTm-OG-oRZSw0ef-V_nY8RsSBgWn1Pb&confirm=likS
Resolving docs.google.com (docs.google.com)... 173.194.218.101, 173.194.218.102, 173.194.218.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.218.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-a8-docs.googleusercontent.com/docs/securesc/01br0merpc7mlcqf4ivodbb88pd2d7sp/8vptdb

In [5]:
!unzip /content/CWE01.zip

Archive:  /content/CWE01.zip
  inflating: w2v_CNA_ASBC_300d.vec   


In [15]:
%%time
from gensim.models import KeyedVectors
#model.wv.vocab

#gensim.models.keyedvectors.Word2VecKeyedVectors
model = KeyedVectors.load_word2vec_format('/content/w2v_CNA_ASBC_300d.vec' 
                                          ,binary=False, encoding='utf-8', unicode_errors='ignore')

CPU times: user 2min 25s, sys: 4.37 s, total: 2min 30s
Wall time: 2min 26s


In [28]:
del_set = {'經營','小時','信義'}

### Cosine Search

In [16]:
from collections import deque
from itertools import compress

#iput pickle file , return filted list
def w2v_filtered_func(pkl_obj, gensim_vec, del_set, top_n, keep_threshold):
    '''
    pkl_obj: brand實體物件(pickle)
    gensim_vec: 詞向量(vec)
    del_set: 贅字集合(set)
    top_n: 每個詞取與之simularity前n大的詞, 做累加計算
    keep_threshold: 保留累加quantile數列中, 大於keep_threshold的詞
    '''
    #不在贅字典的才計算
    l = [x[0] for x in pkl_obj.freq_list if x[0] not in del_set]
    #有可能清完有些是空的list: 直接return
    if l == []: 
        pkl_obj.tagging_list_w2v = []
        return pkl_obj
    tmp_l = deque(l); s = set()
    tmp_sim = np.zeros(len(l)) #紀錄投票分數
    
    #例外處理:如果我想取前幾大的比原本的Attribute長, 就只用原本的Attribute長度
    if top_n>len(l):
        top_n=len(l)

    while tmp_l:        
        tmp = tmp_l.popleft()
        print(f"/*--當前變數:{tmp}--*/")
        print(f"--各simularity的名字:--\n {l}")
        print(f"--紀錄simularity的array:--\n {tmp_sim}")

        if tmp not in s :
            s.add(tmp)
        else: break

        dic = {}
        for x in tmp_l :
            #計算相似度, 記錄在dictionary裡
            try:
                dic[x] = gensim_vec.similarity(tmp,x)
            except:
                continue

        #找出前幾大的        
        sorted_list = sorted(dic.items(), key=lambda value: value[1], reverse=True)[:top_n]
        print(f"標籤:{tmp} / 最相關的:{sorted_list}")

        #把前幾大的, 紀錄在array裡
        for item in sorted_list:
            tmp_sim[l.index(item[0])]+=item[1]

        #最後貼回去
        print(f'----{tmp}結束----\n')
        tmp_l.append(tmp)
    
    pkl_obj.tagging_list_w2v = list(set(compress(l, tmp_sim>np.quantile(tmp_sim,keep_threshold))))
    return pkl_obj

In [29]:
%%time
for x in success_list:
    try:
        globals()[f'{x}'] = w2v_filtered_func(eval(x),model,del_set,5,0.1)
    except: 
        print(f'Fail:{x}')

/*--當前變數:書店--*/
--各simularity的名字:--
 ['書店', '風格', '誠品', '城市', '臺灣', '文具', '創意', '設計', '生活', '藝術', '人文', '臺北', '文化', '百貨', '模式', '分店', '商品', '基隆', '閱讀', '核心', '書籍', '敦南店']
--紀錄simularity的array:--
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
標籤:書店 / 最相關的:[('誠品', 0.7858688), ('敦南店', 0.6584639), ('書籍', 0.6436716), ('閱讀', 0.53785324), ('分店', 0.5345786)]
----書店結束----

/*--當前變數:風格--*/
--各simularity的名字:--
 ['書店', '風格', '誠品', '城市', '臺灣', '文具', '創意', '設計', '生活', '藝術', '人文', '臺北', '文化', '百貨', '模式', '分店', '商品', '基隆', '閱讀', '核心', '書籍', '敦南店']
--紀錄simularity的array:--
 [0.         0.         0.78586882 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.53457862 0.         0.
 0.53785324 0.         0.64367157 0.6584639 ]
標籤:風格 / 最相關的:[('藝術', 0.54718137), ('創意', 0.5229916), ('文化', 0.44801313), ('人文', 0.4416786), ('設計', 0.42979792)]
----風格結束----

/*--當前變數:誠品--*/
--各simularity的名字:--
 ['書店', '風格', '誠品', '城市', '臺灣'

### Save to csv

In [30]:
name=[]; 
tag_list=[]; tag_list_w2v=[]; tag_list_diff=[]

for x in success_list:
    try:
        tmp = eval(x)
        name.append(tmp.name)
        tag_list.append(tmp.tagging_list)
        tag_list_w2v.append(tmp.tagging_list_w2v)
        tag_list_diff.append(set(tmp.tagging_list) - set(tmp.tagging_list_w2v))
    except:
        print(f"Fail:{x}")

In [31]:
import pandas as pd
df = pd.DataFrame(list(zip(name,tag_list,tag_list_w2v,tag_list_diff)),
                 columns=['Brand_Name','Attribute_Freq15','Attribute_Freq15_w2v_filter','Attribute_diff'])

In [32]:
df

,Brand_Name,Attribute_Freq15,Attribute_Freq15_w2v_filter,Attribute_diff
0,誠品書店,"[書店, 風格, 誠品, 城市, 臺灣, 文具, 創意, 設計, 生活, 藝術, 人文, 臺...","[設計, 文具, 敦南店, 臺灣, 人文, 書店, 文化, 生活, 風格, 閱讀, 藝術, ...","{經營, 基隆, 城市, 小時, 信義, 核心}"
1,金石堂,"[內容, 書店, 金石堂, 美食, 書籍, 書局, 網路, 經營, 臺灣, 商品, 門市, ...","[美食, 實體, 門市, 誠品, 書局, 商品, 內容, 書店, 蛋糕, 書籍, 金石堂, ...","{經營, 臺灣, 博客來}"
2,The Wall LiveHouse,"[音樂, 臺灣, 樂團, 地點, 搖滾, 演出, 臺北, 公館, 臺北市, 羅斯福路, 樂迷]","[臺北, 搖滾, 臺北市, 地點, 音樂, 羅斯福路, 樂迷, 樂團, 演出]","{臺灣, 公館}"
3,星巴克,"[咖啡, 星巴克, 設計, 臺北, 門市, 打造, 臺中, 建築, 文化, 整理, 臺灣, ...","[臺北, 咖啡, 門市, 設計, 世界, 空間, 景點, 全球, 顧客, 臺灣, 打造, 飲...","{臺中, 整理}"


In [33]:
for x in df.values:
  print(f'Brand: {x[0]}')
  print(f'Attribute_Freq15: {x[1]}')
  print(f'Attribute_Freq15_w2v_filter: {x[2]}')
  print(f'Attribute_diff: {x[3]}')
  print('------\n')

Brand: 誠品書店
Attribute_Freq15: ['書店', '風格', '誠品', '城市', '臺灣', '文具', '創意', '設計', '生活', '藝術', '人文', '臺北', '文化', '百貨', '模式', '分店', '經營', '小時', '信義', '商品', '基隆', '閱讀', '核心', '書籍', '敦南店']
Attribute_Freq15_w2v_filter: ['設計', '文具', '敦南店', '臺灣', '人文', '書店', '文化', '生活', '風格', '閱讀', '藝術', '創意', '商品', '分店', '臺北', '百貨', '誠品', '書籍', '模式']
Attribute_diff: {'經營', '基隆', '城市', '小時', '信義', '核心'}
------

Brand: 金石堂
Attribute_Freq15: ['內容', '書店', '金石堂', '美食', '書籍', '書局', '網路', '經營', '臺灣', '商品', '門市', '實體', '評價', '博客來', '推薦', '誠品', '蛋糕']
Attribute_Freq15_w2v_filter: ['美食', '實體', '門市', '誠品', '書局', '商品', '內容', '書店', '蛋糕', '書籍', '金石堂', '網路', '推薦', '評價']
Attribute_diff: {'經營', '臺灣', '博客來'}
------

Brand: The Wall LiveHouse
Attribute_Freq15: ['音樂', '臺灣', '樂團', '地點', '搖滾', '演出', '臺北', '公館', '臺北市', '羅斯福路', '樂迷']
Attribute_Freq15_w2v_filter: ['臺北', '搖滾', '臺北市', '地點', '音樂', '羅斯福路', '樂迷', '樂團', '演出']
Attribute_diff: {'臺灣', '公館'}
------

Brand: 星巴克
Attribute_Freq15: ['咖啡', '星巴克', '設計', '臺北', '門市', '打造', '臺中', '建築', '文